In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

#current_date = pd.to_datetime(datetime.utcnow()).floor('H') - timedelta(hours=1)
current_date = pd.Timestamp('2024-01-04 02:00:00')
print(f'{current_date=}')

current_date=Timestamp('2024-01-04 02:00:00')


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009
Connected. Call `.close()` to terminate connection gracefully.
2024-01-09 01:41:55,946 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009
Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hive (101.52s) 
ts_data=               pickup_hour  rides  pickup_location_id      pickup_ts
17454  2023-12-07 02:00:00      0                   1  1701914400000
128    2023-12-07 03:00:00      0                   1  1701918000000
88301  2023-12-07 04:00:00      1                   1  1701921600000
84087  2023-12-07 05:00:00      2                   1  1701925200000
17527  2023-12-07 06:00:00      0                   1  1701928800000
...                    ...    ...                 ...            ...
184008 2024-01-03 21:00:00      4                 265  1704315600000
182418 2024-01-03 22:00:00      4                 265  1704319200000
185126 2024-01-03 23:00:00      4                 265  1704322800000
182755 2024-01-04 00:00:00      2                 265  1704326400000
185079 2024-01-04 01:00:00      1                 265  1704330000000

[178080 rows x 4 columns]
178080
178080


In [4]:
from src.inference import(
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.5318334381337773, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5318334381337773
[LightGBM] [Warning] bagging_fraction is set=0.8255986967378244, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8255986967378244


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,0.0,2024-01-04 02:00:00
1,2,0.0,2024-01-04 02:00:00
2,3,0.0,2024-01-04 02:00:00
3,4,2.0,2024-01-04 02:00:00
4,5,0.0,2024-01-04 02:00:00
...,...,...,...
260,261,1.0,2024-01-04 02:00:00
261,262,2.0,2024-01-04 02:00:00
262,263,12.0,2024-01-04 02:00:00
263,264,8.0,2024-01-04 02:00:00


Save these predictions in the feature store, so that they can be consumed by Streamlit app

In [8]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_PREDICTIONS_METADATA.name,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/79009/fs/78928/fg/398693


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/79009/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f47dfdcc5b0>, None)